# TRABALHO 3

Modelação e implementação em SMT de um autómato híbrido que descreva o sistema de travagem ABS e verifique as suas propriedades dinâmicas.
O sistema usa 2 variáveis contínuas para descrever a  `velocidade do veículo` em relação ao solo e a `velocidade linear dos pneus` também em relação ao solo. O sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades. 
Como primeira abordagem a componente discreta do sistema contém os seguintes modos:
`Start`, `Free`, `Stopping`, `Blocked`, e `Stopped`.



## Autómatos híbridos


*Autómatos híbridos* são modelos de sistemas ciber-físicos. Essencialmente um autómato híbrido é uma *máquina de estados finita*, onde cada estado (designado por *modo de funcionamento*) descreve o comportamento contínuo de um sistema dinâmico modelado por *relações diferenciais ordinárias* (nas variáveis contínuas e nas suas derivadas em relação ao tempo) codificadas num predicado designado por *flow*. Estas variáveis contínuas evoluem num modo de funcionamento enquanto o seu *flow* for válido.
Cada transição discreta entre estados é representada por um arco anotado com um predicado (designado *switch* ou *jump*). Uma transição realiza-se quando o seu *switch* é válido. Ao ocorrer uma transição as variáveis internas dos modos conservam o seu valor, a não ser que lhes seja explicitamente atribuído um novo valor.

Um autómato híbrido pode ser descrito por um FOTS (sobre o qual podemos verificar propriedades lógicas com as metodologias que já estudamos). Nesse processo o FOTS vai *discretizar* as relações diferenciais ordinárias e vai mapea-las num espaço de estados também discreto.

### 1.  Defina um autómato híbrido que descreva a dinâmica do sistema de travagem ABS.

INSERIR IMAGEM E CONDIÇOES

### 2. Modele em lógica temporal LT  propriedades que, na sua opinião, caracterizam o comportamento desejável do sistema. Nomeadamente (mas não só”) a propriedade   ”o veículo para em menos de $t$ segundos” e “a velocidade $V$ diminui sempre com o tempo”.

INSERIR IMAGEM E CONDIÇOES

### 3. Codifique em SMT’s o modelo que definiu em 1.

In [1]:
from z3 import *

Comecemos por declarar os modos.
Na codificação em Z3 é conveniente usar um tipo enumerado para implementar os modos:

In [2]:
Mode,(Start, Free, Stopping, Blocked, Stopped) = EnumSort('Mode',('Start', 'Free', 'Stopping', 'Blocked', 'Stopped'))

Podemos agora declarar as variáveis do FOTS correspondente ao sistema ABS da seguinte forma:

In [3]:
def declare(i):
    s ={}
    s['M'] = Const('M'+str(i), Mode)    # constante do tipo mode
    s['V'] = Real('V'+str(i))           # Velocidade do veículo
    s['v'] = Real('v'+str(i))           # velocidade da roda
    s['F'] = Real('F'+str(i))           # Força F
    s['f'] = Real('f'+str(i))           # força f
    s['P'] = Int('P'+str(i))            # peso
    s['a'] = Real('a'+str(i))           # força de atrito
    s['c'] = Real('c'+str(i))           # valor c
    s['T'] = Real('T'+str(i))           # varíavel contínua que denota o tempo
    return s

De seguida iremos codificar os predicados Z3 `init`, `trans` e `inv`, que caracterizam, respectivamente, os estados iniciais, as transições e o invariante de modo do FOTS correspondente ao sistema ABS.

In [4]:
def init(s):
    return And(s['M'] == Start, s['V'] == 200, s['v'] == 150, s['F'] == s['c'] * (s['V'] - s['v']), \
               s['a'] == 0.4, s['P'] == 100, s['f'] == s['a'] * s['P'], s['c'] == 2, s['T'] == 0)

def inv(s):
    return And(s['V'] - s['v'] >= 0, s['V'] >= 0, s['v'] >= 0)

In [5]:
def trans_timed(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0,p['F'] == s['c'] * (s['V'] - s['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    constante_P = And(s['P'] == 100, p['P'] == s['P'])
    constante_a = And(s['a'] == 0.4, p['a'] == s['a'])
    # Equações com tempo (ao adicionar o tempo, deixam de ser comuns e apenas ocorrem nos loops):
    equacao_V = And(p['V'] - s['V'] == (-s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']))
    equacao_v = And(p['v'] - s['v'] == (-s['a'] * s['P'] + s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']))
    
    # proposições comuns
    props_comuns = And(equacao_F, equacao_f, constante_P, constante_a)
    erro = 1
    
    # timed
    infree = And(s['M'] == Free, \
                 p['M'] == s['M'], \
                 props_comuns, equacao_V, equacao_v, \
                 p['T'] > s['T'])
    instopping = And(s['M'] == Stopping, \
                     p['M'] == s['M'], \
                     props_comuns, equacao_V, equacao_v, \
                     p['T'] > s['T'])
    inblocked = And(s['M'] == Blocked, \
                    p['M'] == s['M'], \
                    props_comuns, equacao_V, equacao_v, \
                    p['T'] > s['T'])
    
    # untimed
    startTofree = And(s['M'] == Start, \
                      p['M'] == Free, \
                      props_comuns, \
                      p['T'] == s['T'])
    freeTostopping = And(s['M'] == Free, s['c'] > 0, s['v'] > 0, s['V'] > 0, \
                         p['M'] == Stopping, p['c'] == s['c'], p['v'] == s['v'], p['V'] == s['V'], \
                         props_comuns, p['T'] == s['T'])
    stoppingToblocked = And(s['M'] == Stopping, \
                            s['c'] == 10, p['M'] == Blocked, \
                            p['c'] == 2, props_comuns, p['T'] == s['T'], p['v'] == s['v'], p['V'] == s['V'])
    stoppingTostopped = And(s['M'] == Stopping, s['c'] == 10, \
                            p['M'] == Stopped, p['v'] == s['v'], p['V'] == s['V'], p['c'] == 0, \
                            props_comuns, p['T'] == s['T'])
    blockedTofree = And(s['M'] == Blocked, s['v'] >= 0, s['V'] - s['v'] > erro, s['c'] > 0, \
                        p['v'] == s['v'], p['V'] == s['V'], p['M'] == Free, p['c'] == s['c'], \
                        props_comuns, p['T'] == s['T'])
    
    return Or(infree, instopping, inblocked, startTofree, freeTostopping, stoppingToblocked, blockedTofree, \
              stoppingTostopped)

In [6]:
def trans_timed(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0,p['F'] == s['c'] * (s['V'] - s['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    constante_P = And(s['P'] == 100, p['P'] == s['P'])
    constante_a = And(s['a'] == 0.4, p['a'] == s['a'])
    # Equações com tempo (ao adicionar o tempo, deixam de ser comuns e apenas ocorrem nos loops):
    equacao_V = And(p['V'] - s['V'] == (-s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']))
    equacao_v = And(p['v'] - s['v'] == (-s['a'] * s['P'] + s['c'] * (s['V'] - s['v'])) * (p['T'] - s['T']))
    
    # proposições comuns
    props_comuns = And(equacao_F, equacao_f, constante_P, constante_a)
    erro = 1
    
    # timed
    infree = And(s['M'] == Free, s['V'] >= 0, s['v'] >= 0, s['c'] == 2, \
                 p['M'] == s['M'], p['c'] == s['c'], \
                 props_comuns, equacao_V, equacao_v, \
                 p['T'] == s['T'] + 1)
    
    instopping = And(s['M'] == Stopping, s['V'] >= 0, s['v'] >= 0, s['c'] == 20, \
                     p['M'] == s['M'], p['c'] == s['c'], \
                     props_comuns, equacao_V, equacao_v, \
                     p['T'] == s['T'] + 1)
    
    inblocked = And(s['M'] == Blocked, \
                    p['M'] == s['M'], s['V'] == s['v'], s['c'] == 2, \
                    props_comuns, equacao_V, equacao_v, p['c'] == s['c'], \
                    p['T'] == s['T'] + 1)
    
    # untimed
    startTofree = And(s['M'] == Start, s['V'] >= 0, s['v'] >= 0, s['c'] == 2, \
                      p['M'] == Free, p['V'] == s['V'], p['v'] >= s['v'], p['c'] == s['c'], \
                      props_comuns, p['T'] == s['T'])
    freeTostopping = And(s['M'] == Free, s['v'] > 0, s['V'] > 1.5 * s['v'], s['c'] == 2, \
                         p['M'] == Stopping, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 20, \
                         props_comuns, \
                         p['T'] == s['T'])
    stoppingToblocked = And(s['M'] == Stopping, s['c'] == 20, \
                            p['M'] == Blocked, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 2,\
                            props_comuns, \
                            p['T'] == s['T'])
    stoppingTostopped = And(s['M'] == Stopping, s['c'] == 20, s['v'] <= 1, \
                            p['M'] == Stopped, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 2, \
                            props_comuns, \
                            p['T'] == s['T'])
    blockedTofree = And(s['M'] == Blocked, s['c'] == 2, \
                        p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'],  p['c'] == s['c'],\
                        props_comuns, \
                        p['T'] == s['T'])
    
    return Or(infree, instopping, inblocked, startTofree, freeTostopping, stoppingToblocked, blockedTofree, \
              stoppingTostopped)

In [7]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [8]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [9]:
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################
##############################################################################################################

In [12]:
def trans(s, p):
    # proposições lógicas comuns entre os vários predicados
    equacao_F = And(s['F'] >= 0, p['F'] == s['c'] * (s['V'] - s['v']))
    equacao_f = And(s['f'] >= 0, p['f'] == s['a'] * s['P'])
    constante_P = And(s['P'] == 100, p['P'] == s['P'])
    constante_a = And(s['a'] == 0.4, p['a'] == s['a'])
    
    # proposições comuns
    props_comuns = And(equacao_F, equacao_f, constante_P, constante_a)
    erro = .654
    
    # self-loops
    infree = And(s['M'] == Free, s['V'] > 0, s['v'] > 0, s['c'] == 2, \
                 p['M'] == s['M'], p['V'] < s['V'], p['v'] < s['v'], p['c'] > 0, \
                 props_comuns)
    instopping = And(s['M'] == Stopping, s['V'] > s['v'], s['v'] > 0, s['c'] == 10, \
                     p['M'] == s['M'], p['V'] < s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                     props_comuns)
    inblocked = And(s['M'] == Blocked, s['V'] > 0, s['v'] == s['V'], s['c'] > 0, s['c'] < 2, \
                    p['M'] == s['M'], p['V'] < s['V'], p['v'] < s['v'], p['V'] > 0, p['c'] == s['c'], \
                    props_comuns)
    
    # transições entre estados
    startTofree = And(s['M'] == Start, s['c'] > 0, \
                      p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], props_comuns)
    freeTostopping = And(s['M'] == Free, s['V'] > s['v'] * 2, s['v'] > 0, s['c'] > 0, \
                         p['M'] == Stopping, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                         props_comuns)
    stoppingToblocked = And(s['M'] == Stopping, s['v'] > 0, s['V'] - s['v'] <= erro, s['V'] > 80, 
                            s['c'] == 10, p['M'] == Blocked, p['V'] == s['V'], p['v'] == s['v'],\
                            p['c'] == 2, props_comuns)
    stoppingTostopped = And(s['M'] == Stopping, s['v'] > 0, s['v'] <= 1, s['V'] == s['v'], s['c'] == 10, \
                            p['M'] == Stopped, p['V'] == s['V'], p['v'] == s['v'], p['c'] == 0, \
                            props_comuns)
    blockedTofree = And(s['M'] == Blocked, s['v'] >= 0, s['V'] - s['v'] > erro, s['c'] > 0, \
                        p['M'] == Free, p['V'] == s['V'], p['v'] == s['v'], p['c'] == s['c'], \
                        props_comuns)
    
    return Or(infree, instopping, inblocked, startTofree, freeTostopping, stoppingToblocked, blockedTofree, \
              stoppingTostopped)

In [13]:
def gera_traco(declare,init,trans,inv,k):
    s = Solver()
    
    #criar k cópias do estado, guardar na lista do traço
    trace = []
    for i in range(k):
        trace.append(declare(i))
        
    #restriçoes init e trans    
    s.add(init(trace[0]))
    
    for i in range(k):
        s.add(inv(trace[i]))
    
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i + 1]))

    print(s.check())
    
    if s.check() == sat:
        m = s.model()
        for i in range(k):
            print(i)
            for v in trace[i]:
                if trace[i][v].sort() != RealSort():
                    print(v, '=', m[trace[i][v]])
                else:
                    r = m[trace[i][v]]
                    f = float(r.numerator_as_long())/float(r.denominator_as_long())
                    print(v, '=', f)
                
#gera_traco(declare, init, trans, inv, 50)

gera_traco(declare, init, trans_timed, inv, 20)

sat
0
M = Start
V = 200.0
v = 150.0
F = 100.0
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 0.0
1
M = Free
V = 200.0
v = 189.97147558968732
F = 100.0
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 0.0
2
M = Free
V = 179.94295117937466
v = 170.02852441031268
F = 20.057048820625344
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 1.0
3
M = Free
V = 160.11409764125068
v = 149.85737794843664
F = 19.828853538123973
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 2.0
4
M = Free
V = 139.6006582556226
v = 130.37081733406472
F = 20.513439385628086
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 3.0
5
M = Free
V = 121.14097641250686
v = 108.83049917718047
F = 18.459681843115742
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 4.0
6
M = Free
V = 96.52002194185408
v = 93.45145364783325
F = 24.62095447065277
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 5.0
7
M = Free
V = 90.3828853538124
v = 59.58859023587493
F = 6.13713658804169
f = 40.0
P = 100
a = 0.4
c = 2.0
T = 6.0
8
M = Stopping
V = 90.3828853538124
v = 59.58859023587493
F = 61.58859023587493
f = 40.0
P = 100
a = 